# The Julia Package manager

A key strength of Julia is that it is easy to reuse other peoples code. To facilitate this, Julia includes a package manager, named Pkg. Here are some of the things it does:

- adding and removing packages
- create project environments using Project.toml
- allow fully reproducible projects using Manifest.toml
- specifying compatibility with dependencies
- facilitate developing packages

## Links
- Pkg documentation - https://julialang.github.io/Pkg.jl/v1/
- BinaryBuilder, package binary dependencies - https://binarybuilder.org/
- Package compiler, bundle relocatable projects - https://julialang.github.io/PackageCompiler.jl/dev/
- General package registry - https://github.com/JuliaRegistries/General/
- JuliaHub (beta), browse available packages - https://juliahub.com/
- Julia Packages, browse available packages - https://juliapackages.com/

In [1]:
versioninfo()

Julia Version 1.7.3
Commit 742b9abb4d (2022-05-06 12:58 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin21.4.0)
  CPU: Apple M1 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, westmere)
Environment:
  JULIA_NUM_THREADS = 8


## Adding and removing packages

The most convenient way to use the package manager is to use the Pkg REPL-mode. To enter the Pkg REPL-mode in an interactive session, type `]` in the prompt. The prompt indicator will go from `julia>` to `pkg>`. In a Jupyter notebook, just start the cell with `]` for the same effect. Remember that with `]?` you can always get help on the Pkg REPL-mode, and `]?add` for specific help for `add`, or any other command.

In [2]:
]?

  Welcome to the Pkg REPL-mode. To return to the julia> prompt, either press
  backspace when the input line is empty or press Ctrl+C.

  Full documentation available at https://pkgdocs.julialang.org/

  Synopsis

  pkg> cmd [opts] [args]

  Multiple commands can be given on the same line by interleaving a ; between
  the commands. Some commands have an alias, indicated below.

  Commands

  activate: set the primary environment the package manager manipulates

  add: add packages to project

  build: run the build script for packages

  develop, dev: clone the full package repo locally for development

  free: undoes a pin, develop, or stops tracking a repo

  gc: garbage collect packages not used for a significant time

  generate: generate files for a new project

  help, ?: show this message

  instantiate: downloads all the dependencies for the project

  pin: pins the version of packages

  precompile: precompile all the project dependencies

  redo: redo the latest change to the

You can add packages to your current environment using `]add Example`. It is also possible to add multiple packages at the same time, or specify a desired version. By default it will add the most recent version that is compatible with the packages that are already installed.

In [3]:
]add Example UnicodePlots

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/code/Project.toml`
  No Changes to `~/code/Manifest.toml`


When packages are added to an environment, they can directly be used. The first time a (new version of a) package is loaded, it will precompile the code, to make it faster to load in the future. If you want, you can also force this to happen for all packages using `]precompile`.

In [4]:
using UnicodePlots

[UnicodePlots](https://github.com/Evizero/UnicodePlots.jl) is a fun package, which allows making plots composed entirely of text characters. We can try it out by generating 10 thousand normally distributed random numbers, and making a histogram.

In [5]:
histogram(randn(10_000))

                ┌                                        ┐ 
   [-4.0, -3.5) ┤▏ 4                                       
   [-3.5, -3.0) ┤▍ 15                                      
   [-3.0, -2.5) ┤▊ 46                                      
   [-2.5, -2.0) ┤██▊ 164                                   
   [-2.0, -1.5) ┤███████▋ 447                              
   [-1.5, -1.0) ┤███████████████▊ 912                      
   [-1.0, -0.5) ┤██████████████████████████▎ 1514          
   [-0.5,  0.0) ┤█████████████████████████████████▏ 1913   
   [ 0.0,  0.5) ┤██████████████████████████████████  1967  
   [ 0.5,  1.0) ┤█████████████████████████▊ 1492           
   [ 1.0,  1.5) ┤███████████████▎ 876                      
   [ 1.5,  2.0) ┤███████▋ 439                              
   [ 2.0,  2.5) ┤██▊ 160                                   
   [ 2.5,  3.0) ┤▋ 41                                      
   [ 3.0,  3.5) ┤▎ 8                                       
   [ 3.5,  4.0) ┤▏ 2                    

Note that packages can rely on binaries. This is integrated into the design of Pkg, to provide a smooth installation experience. So, for example, people installing the Julia wrapper for the GDAL library, will automatically download a compatible GDAL installation compiled for their system. This is not installed globally, so will not interfere with the rest of the system. See https://binarybuilder.org/ to learn more about how this system works.

## Working with environments

When collaborating on a project, you want it to work the same on everyones computer. However, the default environment probably looks different for everybody, depending on the packages they have installed. This can mean not everyone has the same version of packages your project depends on. Not great for reproducible science!

Luckily Julia's package manager has the concept of environments. They can be created using `]activate path`, where path is the directory where the project is located, so `]activate .` creates an environment in the current directory.

In [6]:
]activate MyJuliaProject

  Activating project at `~/code/julia-pizza-course/MyJuliaProject`


Now you can add packages that your project relies on. Note that if the same version of the package is already installed on your computer, for instance in another environment, it will use the same copy, to save space.

In [7]:
]add Example

   Resolving package versions...
  No Changes to `~/code/julia-pizza-course/MyJuliaProject/Project.toml`
  No Changes to `~/code/julia-pizza-course/MyJuliaProject/Manifest.toml`


To see the packages added to your current environment, use `]status`, or use `]status Example` to only display a single package.

In [8]:
]status

      Status `~/code/julia-pizza-course/MyJuliaProject/Project.toml`
  [7876af07] Example v0.5.3


The state of your environment is captured in two files, the `Project.toml` and `Manifest.toml` files. We can print out their contents below.

In [9]:
println(read("MyJuliaProject/Project.toml", String))

[deps]
Example = "7876af07-990d-54b4-ab0e-23690620f79a"



In [10]:
println(read("MyJuliaProject/Manifest.toml", String))

# This file is machine-generated - editing it directly is not advised

julia_version = "1.7.3"
manifest_format = "2.0"

[[deps.Example]]
git-tree-sha1 = "46e44e869b4d90b96bd8ed1fdcf32244fddfb6cc"
uuid = "7876af07-990d-54b4-ab0e-23690620f79a"
version = "0.5.3"



In short, the `Project.toml` file lists the dependencies under a `[deps]` section. It also supports other entries that can be added manually, for instance to add metadata or specify compatibility. The `Manifest.toml` specifies exactly which versions of both direct and indirect dependencies are used. If you share a `Manifest.toml` with someone, and someone uses `]instantiate` on it, they will be able to run your project with exactly the same dependencies.

More information can be found in these chapters of the documentation:
- https://julialang.github.io/Pkg.jl/v1/environments/
- https://julialang.github.io/Pkg.jl/v1/toml-files/
- https://julialang.github.io/Pkg.jl/v1/compatibility/

## Binary packages
We stand on the shoulders of giants. This goes for most research, but also for our geospatial stack. Everyone uses GDAL, GEOS and PROJ directly, or indirectly. These C(++) packages need to be compiled to work on our systems and this has often been difficult. In Julia, we've tried to solve this, much like conda does, but more built into the language itself.

In [2]:
]add GDAL

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/code/Project.toml`
  [add2ef01] + GDAL v1.4.0
    Updating `~/code/Manifest.toml`
  [add2ef01] + GDAL v1.4.0
  [2e619515] + Expat_jll v2.4.8+0
  [a7073274] + GDAL_jll v300.500.1+0
  [d604d12d] + GEOS_jll v3.11.0+0
  [aacddb02] + JpegTurbo_jll v2.1.2+0
  [88015f11] + LERC_jll v3.0.0+1
  [89763e89] + Libtiff_jll v4.4.0+0
  [d3a379c0] + LittleCMS_jll v2.12.0+0
  [643b3616] + OpenJpeg_jll v2.4.0+0
  [58948b4f] + PROJ_jll v900.0.100+1
  [76ed43ae] + SQLite_jll v3.39.2+0
  [3161d3a3] + Zstd_jll v1.5.2+0
  [06c338fa] + libgeotiff_jll v1.7.1+0
  [b53b4c65] + libpng_jll v1.6.38+0


In [6]:
] test GDAL

     Testing GDAL
      Status `/private/var/folders/w_/152mmjvx4cx7y55ky886tp1h0000gn/T/jl_YHGf2u/Project.toml`
  [fa961155] CEnum v0.4.2
  [add2ef01] GDAL v1.4.0
  [a7073274] GDAL_jll v300.500.1+0
  [58948b4f] PROJ_jll v900.0.100+1
  [ca575930] NetworkOptions `@stdlib/NetworkOptions`
  [8dfed614] Test `@stdlib/Test`
      Status `/private/var/folders/w_/152mmjvx4cx7y55ky886tp1h0000gn/T/jl_YHGf2u/Manifest.toml`
  [fa961155] CEnum v0.4.2
  [add2ef01] GDAL v1.4.0
  [692b3bcd] JLLWrappers v1.4.1
  [21216c6a] Preferences v1.3.0
  [2e619515] Expat_jll v2.4.8+0
  [a7073274] GDAL_jll v300.500.1+0
  [d604d12d] GEOS_jll v3.11.0+0
  [aacddb02] JpegTurbo_jll v2.1.2+0
  [88015f11] LERC_jll v3.0.0+1
  [89763e89] Libtiff_jll v4.4.0+0
  [d3a379c0] LittleCMS_jll v2.12.0+0
  [643b3616] OpenJpeg_jll v2.4.0+0
  [58948b4f] PROJ_jll v900.0.100+1
  [76ed43ae] SQLite_jll v3.39.2+0
  [3161d3a3] Zstd_jll v1.5.2+0
  [06c338fa] libgeotiff_jll v1.7.1+0
  [b53b4c65] libpng_jll v1.6.38+0
  [0dad84c5] ArgTools `@st

Test Summary: | Pass  Total
GDAL          |  125    125


     Testing GDAL tests passed 


## Developing a package

How to create your own package or make changes to someone elses package is out of scope for this tutorial. We can give you some references however.

If you want to make changes to an existing package, you can run `]dev Example` to put the git repository in your home directory under `.julia/dev/Example`. See also the usage example of the Revise package linked below.

- Revise.jl https://timholy.github.io/Revise.jl/stable/
- Look at a simple example package for how to structure code https://github.com/JuliaLang/Example.jl
- Video tutorial on developing Julia packages https://youtu.be/QVmU29rCjaA
- This automates creating packages including testing services and more https://github.com/invenia/PkgTemplates.jl

## Exercise

Use https://juliahub.com/, https://discourse.julialang.org/ or simply a web search to browse and find packages that seem interesting to you, and install them using the package manager.

If you have time, you can use the documentation or GitHub page of a package to try it out.

This is intended simply as an open ended exercise to make you more familiar with discovering and installing packages.

To help, here is a small list of packages that some of you may be interested in:
- [ShallowWaters - hydrodynamic model](https://github.com/milankl/ShallowWaters.jl)
- [Oceananigans - hydrodynamic model](https://github.com/CliMA/Oceananigans.jl)
- [MPI - parallel computing](https://github.com/JuliaParallel/MPI.jl)
- [ArchGDAL - geospatial data](https://github.com/yeesian/ArchGDAL.jl)
- [NCDatasets - NetCDF data](https://github.com/Alexander-Barth/NCDatasets.jl)
- [GeometryBasics - geometry primitives](https://github.com/JuliaGeometry/GeometryBasics.jl)
- [MLJ - machine learning](https://alan-turing-institute.github.io/MLJ.jl/stable/)
- [Turing - probabilistic programming](https://turing.ml/dev/)
- [Flux - deep learning](https://fluxml.ai/)
- [Unitful - track units](https://painterqubits.github.io/Unitful.jl/latest/)
- [Measurements - track uncertainty](https://juliaphysics.github.io/Measurements.jl/stable/)
- [DataAssim - data assimilation](https://github.com/Alexander-Barth/DataAssim.jl)
- [GeoStats - geostatistics](https://github.com/JuliaEarth/GeoStats.jl)
